In [ ]:
# we are doing this in a copy big data ->map frame then this doing practically in a here 
# mechanical enginerring 

import os
os.environ['PYSPARK_PYTHON']='python'


In [ ]:
from pyspark.sql import SparkSession
import re

# Create Spark session
spark = SparkSession.builder.appName("Word_count").getOrCreate()

sc = spark.sparkContext #data structure of spark remember it 

In [ ]:
rom pyspark.sql import SparkSession
import re

# Create Spark session
spark = SparkSession.builder.appName("Word_count").getOrCreate()

sc = spark.sparkContext #data structure of spark remember it 

In [3]:
text  = """Apache Spark is a unified analytics engine for big data processing. Spark provides high level APIs in Java, Scale, Pyhon
and in R. Spark is designed for fast and easy-to-use machine learning."""

In [4]:
from  pyspark.sql.functions import explode, split, lower, col, count

In [5]:
df = spark.createDataFrame([(text,)], ["text"])

In [6]:
df.show()

+--------------------+
|                text|
+--------------------+
|Apache Spark is a...|
+--------------------+



In [ ]:
word_df = (df
           .select(explode(split(lower(col("text")), r"[\s\W]+")).alias("word"))   # r is a regular express, /s is a space 
           .filter(col("word") != ""))

In [9]:
word_df.show()

+----------+
|      word|
+----------+
|    apache|
|     spark|
|        is|
|         a|
|   unified|
| analytics|
|    engine|
|       for|
|       big|
|      data|
|processing|
|     spark|
|  provides|
|      high|
|     level|
|      apis|
|        in|
|      java|
|     scale|
|     pyhon|
+----------+
only showing top 20 rows


In [10]:
word_count_df = word_df.groupBy("word").count().orderBy(col("count").desc())
word_count_df.show()

+---------+-----+
|     word|count|
+---------+-----+
|    spark|    3|
|      for|    2|
|       in|    2|
|       is|    2|
|      and|    2|
|    scale|    1|
|    pyhon|    1|
|    level|    1|
| provides|    1|
|     apis|    1|
|  machine|    1|
|     fast|    1|
| learning|    1|
|      use|    1|
| designed|    1|
|   apache|    1|
|     data|    1|
|     high|    1|
|analytics|    1|
|     easy|    1|
+---------+-----+
only showing top 20 rows


In [ ]:
#method 2 creation of RDDs (Resillient distributed dataset) 
# ----> it is a immutable

rdd = sc.parallelize([text])

In [12]:
words_rdd = rdd.flatMap(lambda line: re.findall(r"\b\w+\b", line.lower()))

In [13]:
pairs_rdd = words_rdd.map(lambda word: (word,1))

In [14]:
word_counts_rdd = pairs_rdd.reduceByKey(lambda a, b: a+b)

In [15]:
sorted_counts = word_counts_rdd.sortBy(lambda x: x[1], ascending=False)

In [17]:
for word, count, in sorted_counts.collect():
    print(f"{word} : {count}")

spark : 3
in : 2
for : 2
and : 2
is : 2
apache : 1
machine : 1
apis : 1
level : 1
a : 1
high : 1
fast : 1
to : 1
analytics : 1
easy : 1
learning : 1
data : 1
r : 1
java : 1
unified : 1
engine : 1
processing : 1
provides : 1
designed : 1
big : 1
pyhon : 1
use : 1
scale : 1
